# Sentiment Analysis

In [11]:
# import necessary packages
from transformers import pipeline, AutoTokenizer
import numpy as np

In [12]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

Device set to use cpu
C:\Users\erina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [13]:
import pandas as pd
from pathlib import Path

file_path = Path("data/processed")
politician = []
type_of_speech = []
file_name= []
text = []

for file in file_path.rglob("*.txt"):
    separated_file_name = str(file).split("\\") # windows

    politician.append(separated_file_name[2])
    type_of_speech.append(separated_file_name[3])
    file_name.append(separated_file_name[4])
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
        text.append(content)

df = pd.DataFrame({
    'politician': politician,
    'type_of_speech': type_of_speech,
    'file_name': file_name,
    'text': text
})
df

,politician,type_of_speech,file_name,text
0,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_2025_press_releases_senator_angela_...,filter press close press filter press releases...
1,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_final_p...,skip to content home about biography voting re...
2,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_process...,skip to content home about biography voting re...
3,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_van_hollen_elfre...,skip to content home about biography voting re...
4,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_wyden_continue_t...,skip to content home about biography voting re...
...,...,...,...,...
894,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_ranking_mem...,
895,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_san_diego_d...,
896,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_slams_trump...,
897,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_statement_o...,


### Hugging Face Model Limitations

Only accepts 512 tokens at a time. . .

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
    "bhadresh-savani/distilbert-base-uncased-emotion"
)

def chunk_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    # break tokens into lists of <=512
    chunked = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    # decode each chunk back to text
    return [tokenizer.decode(chunk) for chunk in chunked]

In [15]:
def classify_long_text(text, classifier):
    chunks = chunk_text(text)

    results = classifier(chunks, truncation=True)

    # results is a list (1 per chunk), each is a list of dicts (1 per label)
    labels = [d["label"] for d in results[0]]

    # collect all chunk scores
    score_matrix = np.array([[d["score"] for d in chunk] for chunk in results])
    averaged_scores = score_matrix.mean(axis=0)

    # return a dict: {label: averaged_score}
    return dict(zip(labels, averaged_scores))

In [17]:
# small batch test
classify_long_text(df.loc[0, "text"], classifier)

Token indices sequence length is longer than the specified maximum sequence length for this model (1070 > 512). Running this sequence through the model will result in indexing errors


{'sadness': np.float64(0.08329954495032628),
 'joy': np.float64(0.32700725831091404),
 'love': np.float64(0.006001367544134458),
 'anger': np.float64(0.5680736837287744),
 'fear': np.float64(0.014618020271882415),
 'surprise': np.float64(0.0010000822173121076)}

In [10]:
df["sentiment_scores"] = df["text"].apply(lambda t: classify_long_text(t, classifier))

Token indices sequence length is longer than the specified maximum sequence length for this model (5412 > 512). Running this sequence through the model will result in indexing errors


IndexError: list index out of range

In [13]:
df.head()

,politician,type_of_speech,file_name,text,sentiment
0,bernie_sanders,bipartisan_and_other_speeches,bipartisan_sanders_institute_2018_processed.txt,you turn to us for voices you will not hear an...,"[[{'label': 'sadness', 'score': 0.031841225922..."
1,bernie_sanders,bipartisan_and_other_speeches,response_trump_address_2025_processed.txt,prepared remarks: sanders delivers livestreame...,"[[{'label': 'sadness', 'score': 0.027466034516..."
2,bernie_sanders,bipartisan_and_other_speeches,speech_trump_threat_democracy_2020_processed.txt,bernie sanders: ( 05:27 ) let me thank all of ...,"[[{'label': 'sadness', 'score': 0.043067689985..."
3,bernie_sanders,partisan_rally_speeches,rally_2020_campaign_brooklyn_processed.txt,"sen. bernie sanders, i-vt., waves to supporter...","[[{'label': 'sadness', 'score': 0.030290134251..."
4,bernie_sanders,partisan_rally_speeches,rally_dnc_2024_processed.txt,bernie sanders ( 00:01 ): thank you. thank you...,"[[{'label': 'sadness', 'score': 0.083833731710..."
